In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
roi = ee.Geometry.Rectangle([113.7393, 29.8642, 115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [5]:
## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('(B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7)/10000',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

def maskSR_reverse(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    image_cloud = img.updateMask(mask.remap([0,1],[1,0]))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image_shadow = img.updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True).remap([0,1],[1,0]))
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    image_outlier = img.updateMask(bands.gt(0).rename('outlier'))
    return ee.ImageCollection([image_cloud,image_shadow,image_outlier]).sum()

# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

In [6]:
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
# k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

def classified_image(img):
    image = maskSR(img).select(bands).classify(trainedClassifier).eq(1).remap([0,1],[1,2]).rename('waterclass').float()
    invalidPixel = maskSR_reverse(img).select('pixel_qa').gt(0).remap([0,1],[1,0]).rename('waterclass').float()
    class_image = ee.ImageCollection([invalidPixel,image]).sum()
    return class_image.set({'system:id':img.get('system:id')})

def waterArea(image):
    classified_image = image.select('waterclass').eq(2).rename('waterclass')
    water_area = classified_image.multiply(ee.Image.pixelArea()).divide(1e6)
    waterarea = water_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': roi,
        'scale': 200,
        'maxPixels': 1e14,
        'tileScale': 2,
    })
    return image.set({'waterarea': waterarea.get('waterclass')})

# img指经过指数计算，但未做云掩膜的图像
def occurrence_Histogram(class_image):
    water = class_image.eq(2).selfMask()
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': roi,
        'scale': 30,
        'bestEffort': True,
        'tileScale': 2,
    })
    return class_image.set({'occurrence_HistogramCount': occurrence_HistogramCount.get('occurrence')})

def AutomaticCorrection_threshold(class_image):
    histogram = ee.List(ee.Dictionary(class_image.get('occurrence_HistogramCount')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(class_image.get('occurrence_HistogramCount')).get('bucketMeans'))
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    return class_image.set({'occurrence_threshold':occurrence_threshold})

# img指经过指数计算(需要校正的图像)，但未做云掩膜的图像
def AutomaticCorrection_enhanced(class_image):
    basemap = ee.Image.constant(0).toFloat().updateMask(class_image.gte(0)).rename('waterclass')
    water = class_image.eq(2).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_no_data = occurrence.updateMask(class_image.eq(0).selfMask())
    occurrence_threshold = class_image.get('occurrence_threshold')
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water

In [7]:
for year in range(2000,2021,1):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)  
    l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)           
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
           .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)
    landsat_image = l8.merge(l7).merge(l5).map(water_index)
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    # 'wuhan'需要修改
    filtered_samples = k_mean(ee.FeatureCollection('users/311605001111/wuhan_tibetan/wuhan_' + str(year)))
    trainedClassifier = ee.Classifier.smileRandomForest(20).train(filtered_samples,label,bands)
    landsat_images = landsat_image.map(classified_image)
    # 两步筛选
    probable_correct_image = landsat_images.map(waterArea).filter(ee.Filter.gt('waterarea',2))
    No_correct_image1 = landsat_images.map(waterArea).filter(ee.Filter.lte('waterarea',2))
    occurrence_threshold = probable_correct_image.map(occurrence_Histogram).filter(ee.Filter.neq('occurrence_HistogramCount',None)).map(AutomaticCorrection_threshold)
    correct_image = occurrence_threshold.filter(ee.Filter.gte('occurrence_threshold',5)).filter(ee.Filter.lt('occurrence_threshold',75))
    No_correct_image2 = occurrence_threshold.filter(ee.Filter.lt('occurrence_threshold',5))
    No_correct_image3 = occurrence_threshold.filter(ee.Filter.gte('occurrence_threshold',75))
    No_correct_image4 = probable_correct_image.map(occurrence_Histogram).filter(ee.Filter.eq('occurrence_HistogramCount',None))
    correct_image = correct_image
    print('correct: ',correct_image.size().getInfo())
    No_correct_image = No_correct_image1.merge(No_correct_image2).merge(No_correct_image3).merge(No_correct_image4)
    print('nocorrect: ',No_correct_image.size().getInfo())
    # 'wuhan'需要修改
    id_VP = 'wuhan_VP_' + str(year)
    ID_VP = 'users/311605001111/WPVP/' + id_VP
    id_WP = 'wuhan_WP_' + str(year)
    ID_WP = 'users/311605001111/WPVP/' + id_WP
    # 水体频率
    if correct_image.size().getInfo() == 0 :
        No_Correct_waterPixel = No_correct_image.map(lambda i : i.select('waterclass').eq(2)).sum()
        No_Correct_validPixel = No_correct_image.map(lambda i : i.select('waterclass').gt(0)).sum()
        task1 = ee.batch.Export.image.toAsset(**{
            'image': No_Correct_waterPixel,
            'description': id_WP,
            'assetId': ID_WP,
            'scale': 30,
            'region': roi,
            'maxPixels': 1e13,
        })
        task1.start()
        task2 = ee.batch.Export.image.toAsset(**{
            'image': No_Correct_validPixel,
            'description': id_VP,
            'assetId': ID_VP,
            'scale': 30,
            'region': roi,
            'maxPixels': 1e13,
        })
        task2.start()
    else:
        correct_waterPixel = correct_image.map(AutomaticCorrection_enhanced).sum()
        correct_validPixel = correct_image.map(lambda i : i.select('waterclass').gte(0)).sum()
        No_Correct_waterPixel = No_correct_image.map(lambda i : i.select('waterclass').eq(2)).sum()
        No_Correct_validPixel = No_correct_image.map(lambda i : i.select('waterclass').gt(0)).sum()
        waterPixel = ee.ImageCollection([correct_waterPixel,No_Correct_waterPixel]).sum()
        validPixel = ee.ImageCollection([correct_validPixel,No_Correct_validPixel]).sum()
        task1 = ee.batch.Export.image.toAsset(**{
            'image': waterPixel,
            'description': id_WP,
            'assetId': ID_WP,
            'scale': 30,
            'region': roi,
            'maxPixels': 1e13,
        })
        task1.start()
        task2 = ee.batch.Export.image.toAsset(**{
            'image': validPixel,
            'description': id_VP,
            'assetId': ID_VP,
            'scale': 30,
            'region': roi,
            'maxPixels': 1e13,
        })
        task2.start()

2000
correct:  5
nocorrect:  65
2001
correct:  9
nocorrect:  80
2002
correct:  1
nocorrect:  80
2003
correct:  0
nocorrect:  79
2004
correct:  3
nocorrect:  90
2005
correct:  4
nocorrect:  72
2006
correct:  14
nocorrect:  66
2007
correct:  10
nocorrect:  65
2008
correct:  16
nocorrect:  64
2009
correct:  11
nocorrect:  71
2010
correct:  3
nocorrect:  66
2011
correct:  16
nocorrect:  57
2012
correct:  6
nocorrect:  27
2013
correct:  21
nocorrect:  76
2014
correct:  17
nocorrect:  70
2015
correct:  21
nocorrect:  89
2016
correct:  0
nocorrect:  109
2017
correct:  9
nocorrect:  97
2018
correct:  17
nocorrect:  87
2019
correct:  12
nocorrect:  85
2020
correct:  3
nocorrect:  82


# 计算水体频率

In [ ]:
# img指经过指数计算(需要校正的图像)，但未做云掩膜的图像
def AutomaticCorrection_enhanced(class_image):
    basemap = ee.Image.constant(0).toFloat().updateMask(class_image.gte(0)).rename('waterclass')
    water = class_image.eq(2).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_no_data = occurrence.updateMask(class_image.eq(0).selfMask())
    occurrence_threshold = class_image.get('occurrence_threshold')
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water

In [ ]:
## 水体频率
# 校正图像
correct_waterPixel = correct_image.map(AutomaticCorrection_enhanced).sum()
correct_validPixel = correct_image.map(lambda i : i.select('waterclass').gte(0)).sum()
# 未校正图像
No_Correct_waterPixel = No_correct_image.map(lambda i : i.select('waterclass').eq(2)).sum()
No_Correct_validPixel = No_correct_image.map(lambda i : i.select('waterclass').gt(0)).sum()

In [ ]:
# 水体频率
waterPixel = ee.ImageCollection([correct_waterPixel,No_Correct_waterPixel]).sum()
Map.addLayer(waterPixel,{'palette':['white','#3dd633'],'min':0,'max':35},"waterPixelCount")
validPixel = ee.ImageCollection([correct_validPixel,No_Correct_validPixel]).sum()
Map.addLayer(validPixel,{'palette':['white','#e40775'],'min':0,'max':35},"validPixel")
waterfrequency = waterPixel.divide(validPixel).rename('frequency')
Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"water frequency")
permanentwater = waterfrequency.gte(0.75)
Map.addLayer(permanentwater.selfMask(),{'palette':['blue']},"permanent water")

In [ ]:
# 水体频率
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.8826, 34.6579,91.3626, 35.8279]
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(waterfrequency, region=region, vis_params=vis,cmap = cmap)
# cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
ax.set_title(label = 'region E/2020', fontsize=28)

In [ ]:
# 永久性水体---v2
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.8826, 34.6579,91.3626, 35.8279]
vis = {'palette':['blue']}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(permanentwater.selfMask(), region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
ax.set_title(label = 'region E/2020', fontsize=28)

In [ ]:
refer_img = l8.map(maskSR).median().clip(roi).visualize(**visParams)
blend = refer_img.blend(permanentwater.selfMask().visualize(**{'palette':['blue']}))
Map.addLayer(blend, {}, "Blend")

# 其他数据集

In [ ]:
year = '2020'
#### 设置年份
JRC_id = 'JRC/GSW1_3/YearlyHistory/' + year
Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + year
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
# JRC
JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
# Maryland
Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()

Map.addLayer(Maryland_waterclass.eq(2).selfMask(),{'palette':['red']},"Maryland permanent")
Map.addLayer(JRC_waterclass.eq(2).selfMask(),{'palette':['green']},"JRC permanent")

In [ ]:
# # JRC permanent waterbody
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [112.7614, 22.2347,114.0514, 23.2547]
vis = {'palette':['D3D3D3','blue'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(JRC_waterclass.eq(2).selfMask(), region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
ax.set_title(label = 'JRC permanent water(PearlRiverDelta/2020)', fontsize=28)

# # Maryland permanent waterbody
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [112.7614, 22.2347,114.0514, 23.2547]
vis = {'palette':['D3D3D3','blue'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(Maryland_waterclass.eq(2).selfMask(), region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
ax.set_title(label = 'Maryland permanent water(PearlRiverDelta/2020)', fontsize=23)

In [ ]:
# landsat_images = l8.map(water_index)
landsat_images = landsat_images
waterPixel = landsat_images.map(lambda i : i.select('waterclass').eq(2)).sum()
validPixel = landsat_images.map(lambda i : i.select('waterclass').gt(0)).sum()

# 水体频率
waterfrequency = waterPixel.select('waterclass').divide(validPixel.select('count')).rename('frequency')
Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"water frequency no")
permanentwater = waterfrequency.gte(0.75)
Map.addLayer(permanentwater.selfMask(),{'palette':['cyan']},"permanent water no")

# 制图

In [ ]:
# 永久性水体---v1
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [115.9244, 38.6272,117.4944,39.6372]
vis = {'palette':['D3D3D3','blue'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(permanentwater, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
ax.set_title(label = 'permanent water(daqing/2020)', fontsize=28)

# 永久性水体---v2
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [112.7614, 22.2347,114.0514, 23.2547]
vis = {'palette':['blue']}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(permanentwater.selfMask(), region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
ax.set_title(label = 'permanent water(PearlRiverDelta/2020)', fontsize=28)

# 水体频率
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [115.9244, 38.6272,117.4944,39.6372]
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(waterfrequency, region=region, vis_params=vis,cmap = cmap)
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
ax.set_title(label = 'waterbody frequency(Auto-Correction/2020)', fontsize=28)

# 参考影像
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [123.9355, 45.8362,125.3055, 46.8762]
fig = plt.figure(figsize=(12, 8))
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}
# use cartoee to get a map
a = l8.filter(ee.Filter.lt('CLOUD_COVER',35)).map(maskSR).median().clip(roi)
ax = cartoee.get_map(a, region=region,vis_params=visParams)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
ax.set_title(label = 'refer image(median composite image/2020)', fontsize=28)

# 合成影像
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee

region = [90.8867, 35.4570,91.3632, 35.7387]
fig = plt.figure(figsize=(12, 8))
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}
# use cartoee to get a map
ax = cartoee.get_map(blend, region=region)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.2,0.1], linestyle=":")
ax.set_title(label = 'Experimental results', fontsize=28)